In [1]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.


In [2]:
import tensorflow as tf
import numpy as np

print(tf.__version__)

2024-08-03 06:05:01.958217: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-03 06:05:01.958314: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-03 06:05:02.104949: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2.15.0


In [3]:
EPOCHS = 10

## DenseUnit 구현
class DenseUnit(tf.keras.Model):
    def __init__(self, filter_out, kernel_size):
        super(DenseUnit, self).__init__()
        # batch normalization -> ReLu -> Conv Layer
        # 여기서 ReLu 같은 경우는 변수가 없는 Layer이므로 여기서 굳이 initialize 해주지 않는다. (call쪽에서 사용하면 되므로)
        # Pre-activation 구조는 똑같이 가져가되, concatenate 구조로 만들어주어야함에 주의하자!!

        self.bn = tf.keras.layers.BatchNormalization()
        self.conv = tf.keras.layers.Conv2D(filter_out, kernel_size, padding='same')
        self.concat = tf.keras.layers.Concatenate()

    def call(self, x, training=False, mask=None): # x : (Batch 갯수, Height, width, Channel_in)
        # training 꼭 잊어버리지 말자!!
        h = self.bn(x, training=training)
        h = tf.nn.relu(h)
        h = self.conv(h) # h : (Batch, height, width, filter_output) zero-padding을 했으므로 filter만 바뀜
        return self.concat([x, h]) # (Batch, height, width, (channel_in + filter_output))

## DenseLayer 구현
class DenseLayer(tf.keras.Model):
    def __init__(self, num_unit, growth_rate, kernel_size):
        super(DenseLayer, self).__init__()
        self.sequence = list()
        for idx in range(num_unit):
            self.sequence.append(DenseUnit(growth_rate, kernel_size))

    def call(self, x, training=False, mask=None):
        for unit in self.sequence:
            x = unit(x, training=training)
        return x

## Transition Layer 구현
# Maxpooling을 해줄 때 필요함.
# 예를 들어, DenseLayer를 사용을 하게 되면, Growth_rate=32, num_unit=8인 경우에는, 32X8 만큼 channel의 수가 급격하게 증가하기 때문에 너무 커질 수 있다. 이럴 경우 이 Transition Layer를 통해 Pooling 전에 channel의 수를 조절해주기 위해 사용되는 것이다.
class TransitionLayer(tf.keras.Model):
    def __init__(self, filters, kernel_size):
        super(TransitionLayer, self).__init__()
        # transition을 할 경우에는 이렇게 convolution을 해서 단순히 filter 개수를 변경만 해준뒤
        # 그 다음에 Maxpooling을 해주는 식으로 구현이 된다.
        self.conv = tf.keras.layers.Conv2D(filters, kernel_size, padding='same')
        self.pool = tf.keras.layers.MaxPool2D()

    def call(self, x, training=False, mask=None):
        # 여기서는 Batch normalization이 없기 때문에 training을 안써줘도 된다.
        x = self.conv(x)
        return self.pool(x)

## 모델 정의
class DenseNet(tf.keras.Model):
    def __init__(self):
        super(DenseNet, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(8, (3, 3), padding='same', activation='relu') # 28x28x8

        # num_unit=2, growth_rate=4, kernel_size=(3,3)
        # num_unit은 ResNet과 동일
        self.dl1 = DenseLayer(2, 4, (3, 3)) # 28x28x(8+2*4)
        self.tr1 = TransitionLayer(16, (3, 3)) # 14x14x16

        self.dl2 = DenseLayer(2, 8, (3, 3)) # 14x14x(16 + 2*8)
        self.tr2 = TransitionLayer(32, (3, 3)) # 7x7x32

        self.dl3 = DenseLayer(2, 16, (3, 3)) # 7x7x(32+2*16)

        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(128, activation='relu')
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax')       

    def call(self, x, training=False, mask=None):
        x = self.conv1(x)

        x = self.dl1(x, training=training)
        x = self.tr1(x)

        x = self.dl2(x, training=training)
        x = self.tr2(x)

        x = self.dl3(x, training=training)

        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)


## 학습, 데스트 루프 정의

# Implement training loop
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images, training=False)

    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [4]:
## 데이터셋 준비
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train[..., tf.newaxis].astype(np.float32)
x_test = x_test[..., tf.newaxis].astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
## 학습 환경 정의
# 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

# 모델 생성
model = DenseNet()

# 손실함수 정의 및 최적화 기법 정의
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# 평가지표 정의
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [7]:
## 학습 루프 동작
# 어떤 것은 DenseNet이 성능이 더 좋게 나오고 어떤 것은 ResNet이 더 좋게 나오기도 한다. 또한 parameter에 따라 다르다. Resnet은 좀 더 안정적이게 수렴하지만 Densenet은 좀 fluctuate하다는 점을 유의하자.

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)

  for test_images, test_labels in test_ds:
    test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)

  template = "Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}"
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result() * 100,
                        test_loss.result(),
                        test_accuracy.result() * 100))

#train_loss.reset_states()
#train_accuracy.reset_states()
#test_loss.reset_states()
#test_accuray.reset_states()

I0000 00:00:1722666035.187315     142 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Epoch 1, Loss: 0.11722613126039505, Accuracy: 96.55166625976562, Test Loss: 0.055627528578042984, Test Accuracy: 98.3499984741211
Epoch 2, Loss: 0.08699080348014832, Accuracy: 97.4566650390625, Test Loss: 0.05324681103229523, Test Accuracy: 98.47999572753906
Epoch 3, Loss: 0.07333599776029587, Accuracy: 97.88166809082031, Test Loss: 0.053093694150447845, Test Accuracy: 98.4566650390625
Epoch 4, Loss: 0.06511535495519638, Accuracy: 98.1312484741211, Test Loss: 0.053559087216854095, Test Accuracy: 98.43999481201172
Epoch 5, Loss: 0.05910111963748932, Accuracy: 98.3169937133789, Test Loss: 0.05311901494860649, Test Accuracy: 98.52400207519531
Epoch 6, Loss: 0.05468367040157318, Accuracy: 98.4477767944336, Test Loss: 0.05416098237037659, Test Accuracy: 98.56832885742188
Epoch 7, Loss: 0.051100026816129684, Accuracy: 98.56071472167969, Test Loss: 0.054947707802057266, Test Accuracy: 98.5971450805664
Epoch 8, Loss: 0.04772491753101349, Accuracy: 98.66020965576172, Test Loss: 0.05793722718954

In [23]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 데이터셋 폴더 경로
dataset_dir = '/kaggle/input/pets-facial-expression-dataset/'

# 데이터 변환 및 로드
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% 데이터를 검증용으로 사용
)

# 학습 데이터 로드
train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(32, 32),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# 검증 데이터 로드
validation_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(32, 32),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# 클래스 이름 및 샘플 수 출력
print(f'Number of classes: {len(train_generator.class_indices)}')
print(f'Class names: {list(train_generator.class_indices.keys())}')
print(f'Number of training images: {train_generator.samples}')
print(f'Number of validation images: {validation_generator.samples}')

# 입력 이미지를 3채널로 변환하는 사용자 정의 데이터 생성기
def custom_generator(generator):
    while True:
        batch_x, batch_y = next(generator)
        yield batch_x, batch_y

Found 1660 images belonging to 5 classes.
Found 414 images belonging to 5 classes.
Number of classes: 5
Class names: ['Angry', 'Master Folder', 'Other', 'Sad', 'happy']
Number of training images: 1660
Number of validation images: 414
Epoch 1/10
51/51 [==============================] - 71s 153ms/step - loss: 1.6180 - accuracy: 0.4743 - val_loss: 1.4994 - val_accuracy: 0.5234
Epoch 2/10
51/51 [==============================] - 10s 94ms/step - loss: 1.4674 - accuracy: 0.4994 - val_loss: 1.3984 - val_accuracy: 0.5156
Epoch 3/10
51/51 [==============================] - 5s 92ms/step - loss: 1.3881 - accuracy: 0.5092 - val_loss: 1.3910 - val_accuracy: 0.5156
Epoch 4/10
51/51 [==============================] - 5s 90ms/step - loss: 1.3412 - accuracy: 0.5221 - val_loss: 1.4413 - val_accuracy: 0.5156
Epoch 5/10
51/51 [==============================] - 5s 92ms/step - loss: 1.3277 - accuracy: 0.5092 - val_loss: 1.3884 - val_accuracy: 0.5078
Epoch 6/10
51/51 [==============================] - 5s 92m

In [24]:
# 모델 정의 및 컴파일
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model = DenseNet121(weights=None, include_top=False, input_shape=(32, 32, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)  # 클래스 수를 동적으로 설정

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(
    custom_generator(train_generator),
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=custom_generator(validation_generator),
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

## 학습 루프 동작
# 어떤 것은 DenseNet이 성능이 더 좋게 나오고 어떤 것은 ResNet이 더 좋게 나오기도 한다. 또한 parameter에 따라 다르다. Resnet은 좀 더 안정적이게 수렴하지만 Densenet은 좀 fluctuate하다는 점을 유의하자.

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)

  for test_images, test_labels in test_ds:
    test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)

  template = "Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}"
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result() * 100,
                        test_loss.result(),
                        test_accuracy.result() * 100))

#train_loss.reset_states()
#train_accuracy.reset_states()
#test_loss.reset_states()
#test_accuray.reset_states()

Epoch 1/10
51/51 [==============================] - 60s 139ms/step - loss: 1.6183 - accuracy: 0.4865 - val_loss: 1.4492 - val_accuracy: 0.5156
Epoch 2/10
51/51 [==============================] - 5s 96ms/step - loss: 1.4237 - accuracy: 0.5092 - val_loss: 1.4405 - val_accuracy: 0.5130
Epoch 3/10
51/51 [==============================] - 5s 93ms/step - loss: 1.4028 - accuracy: 0.5031 - val_loss: 1.4346 - val_accuracy: 0.5156
Epoch 4/10
51/51 [==============================] - 5s 96ms/step - loss: 1.3391 - accuracy: 0.5350 - val_loss: 1.4054 - val_accuracy: 0.5286
Epoch 5/10
51/51 [==============================] - 5s 96ms/step - loss: 1.3500 - accuracy: 0.4982 - val_loss: 1.4161 - val_accuracy: 0.5208
Epoch 6/10
51/51 [==============================] - 5s 103ms/step - loss: 1.3468 - accuracy: 0.5092 - val_loss: 1.4113 - val_accuracy: 0.5104
Epoch 7/10
51/51 [==============================] - 5s 99ms/step - loss: 1.3193 - accuracy: 0.5227 - val_loss: 1.4225 - val_accuracy: 0.5156
Epoch 8/10

ValueError: in user code:

    File "/tmp/ipykernel_34/2489616372.py", line 93, in train_step  *
        predictions = model(images, training=True)
    File "/opt/conda/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/opt/conda/lib/python3.10/site-packages/keras/src/engine/input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 of layer "model_1" is incompatible with the layer: expected shape=(None, 32, 32, 3), found shape=(32, 28, 28, 1)
